In [272]:
import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F

In [126]:
texts=["hola mundo cruel", "me llamo eduardo", "buenas noches a todos"]

tokens = []
vocabulary = set()
for text in texts:
    toks = text.split()
    tokens.append(toks)
    for w in toks:
        vocabulary.add(w)

print(tokens)

unk_token = "<UNK>"
pad_token = '<PAD>'
print(vocabulary)

print(f"Vocabulary length {len(vocabulary) + 2}")

[['hola', 'mundo', 'cruel'], ['me', 'llamo', 'eduardo'], ['buenas', 'noches', 'a', 'todos']]
{'cruel', 'todos', 'llamo', 'a', 'hola', 'eduardo', 'noches', 'me', 'buenas', 'mundo'}
Vocabulary length 12


In [127]:
word2id = {unk_token:0, pad_token:1}
id2word = {0:unk_token, 1:pad_token}

for i, w in enumerate(vocabulary,2):
    word2id[w] = i
    id2word[i] = w

print(word2id)
print(id2word)

vocabulary_size = len(word2id)
print(f"Vocabulary length {vocabulary_size}")

{'<UNK>': 0, '<PAD>': 1, 'cruel': 2, 'todos': 3, 'llamo': 4, 'a': 5, 'hola': 6, 'eduardo': 7, 'noches': 8, 'me': 9, 'buenas': 10, 'mundo': 11}
{0: '<UNK>', 1: '<PAD>', 2: 'cruel', 3: 'todos', 4: 'llamo', 5: 'a', 6: 'hola', 7: 'eduardo', 8: 'noches', 9: 'me', 10: 'buenas', 11: 'mundo'}
Vocabulary length 12


In [128]:
def numeralize(text):
    sent = []
    for w in text:
        if w in vocabulary:
            sent.append(word2id[w])
        else:
            sent.append(0)  # <UNK>
    return sent

In [129]:
text = ["hola", "lalo"]
print(numeralize(text))

[6, 0]


In [130]:
text_numbers = [numeralize(tok) for tok in tokens]
print(text_numbers)

[[6, 11, 2], [9, 4, 7], [10, 8, 5, 3]]


In [131]:
larger = 0
for text in text_numbers:
    if len(text) > larger:
        larger = len(text)

print(f"Largest text of {larger} words")

Largest text of 4 words


In [132]:
def pad_sentence(text):
    if len(text) < larger:
        diff = larger - len(text)
        for i in range(diff):
            text.append(1)  # add token "<PAD>"
    return text

In [133]:
padded_text = [pad_sentence(text) for text in text_numbers]
print(padded_text)

[[6, 11, 2, 1], [9, 4, 7, 1], [10, 8, 5, 3]]


In [134]:
def textify(text):
    words = []
    for number in text:
        words.append(id2word[number])
    return words

In [135]:
text_again = [textify(text) for text in padded_text]
print(text_again)

[['hola', 'mundo', 'cruel', '<PAD>'], ['me', 'llamo', 'eduardo', '<PAD>'], ['buenas', 'noches', 'a', 'todos']]


In [146]:
class Embedder(nn.Module):
    def __init__(self, vocab_size, emb_dim):
        super(Embedder, self).__init__()
        self.embed = nn.Embedding(vocab_size+1, emb_dim)    # +1 for the ids not starting with 0

    def forward(self, x):
        return self.embed(x)

    def print_weights(self):
        return self.embed.weight

In [147]:
embedding_dimension = 5

text1 = padded_text[0]
print(text1)
print(type(text1))

[6, 11, 2, 1]
<class 'list'>


In [148]:
batch = np.vstack(padded_text)
print(batch)
print(type(batch))
print(batch[0])
print(type(batch[0]))

[[ 6 11  2  1]
 [ 9  4  7  1]
 [10  8  5  3]]
<class 'numpy.ndarray'>
[ 6 11  2  1]
<class 'numpy.ndarray'>


In [149]:
input_batch = torch.tensor(batch, dtype=torch.long)
print(input_batch)

tensor([[ 6, 11,  2,  1],
        [ 9,  4,  7,  1],
        [10,  8,  5,  3]])


In [150]:
print(input_batch[0])
print(type(input_batch[0]))

tensor([ 6, 11,  2,  1])
<class 'torch.Tensor'>


In [151]:
embedder = Embedder(vocabulary_size, embedding_dimension)
print(embedder.print_weights())


Parameter containing:
tensor([[-1.5265e+00, -1.4884e+00,  5.3641e-01, -2.4606e-01,  4.5496e-01],
        [-7.7714e-01, -2.0969e+00,  1.7835e+00, -2.6246e+00,  4.4385e-02],
        [-9.0353e-01,  6.0791e-02, -7.1391e-01,  1.5493e+00,  2.6345e-03],
        [-3.7953e-01,  4.6506e-01, -2.4369e+00, -1.8846e-01,  2.1542e+00],
        [ 5.9098e-01,  3.7616e-01, -3.1290e-01,  5.6046e-01,  1.0438e-01],
        [ 1.2133e+00,  1.3757e+00,  1.1258e+00,  8.7801e-01, -1.1513e+00],
        [ 8.0069e-02, -1.1752e+00, -3.9491e-03,  1.1250e-03, -5.0092e-01],
        [-5.6224e-02, -5.7728e-01,  1.3235e-01, -1.6854e+00, -5.7394e-01],
        [-3.2366e-01,  7.3624e-01, -5.7591e-01,  4.0074e-01, -1.5145e+00],
        [-3.0589e-01, -1.0460e-01,  1.0548e+00,  8.8961e-01,  1.1981e+00],
        [ 1.0106e+00,  1.0115e+00,  3.5873e-01,  9.3127e-01,  1.6046e-01],
        [ 5.4267e-01, -1.4515e+00,  5.6311e-01,  3.6198e-01, -8.6320e-01],
        [ 8.8370e-01, -4.5666e-01,  8.0039e-01, -2.6311e+00,  2.1566e-01]],
  

In [152]:
print(embedder(input_batch))
print(embedder(input_batch).shape)


tensor([[[ 8.0069e-02, -1.1752e+00, -3.9491e-03,  1.1250e-03, -5.0092e-01],
         [ 5.4267e-01, -1.4515e+00,  5.6311e-01,  3.6198e-01, -8.6320e-01],
         [-9.0353e-01,  6.0791e-02, -7.1391e-01,  1.5493e+00,  2.6345e-03],
         [-7.7714e-01, -2.0969e+00,  1.7835e+00, -2.6246e+00,  4.4385e-02]],

        [[-3.0589e-01, -1.0460e-01,  1.0548e+00,  8.8961e-01,  1.1981e+00],
         [ 5.9098e-01,  3.7616e-01, -3.1290e-01,  5.6046e-01,  1.0438e-01],
         [-5.6224e-02, -5.7728e-01,  1.3235e-01, -1.6854e+00, -5.7394e-01],
         [-7.7714e-01, -2.0969e+00,  1.7835e+00, -2.6246e+00,  4.4385e-02]],

        [[ 1.0106e+00,  1.0115e+00,  3.5873e-01,  9.3127e-01,  1.6046e-01],
         [-3.2366e-01,  7.3624e-01, -5.7591e-01,  4.0074e-01, -1.5145e+00],
         [ 1.2133e+00,  1.3757e+00,  1.1258e+00,  8.7801e-01, -1.1513e+00],
         [-3.7953e-01,  4.6506e-01, -2.4369e+00, -1.8846e-01,  2.1542e+00]]],
       grad_fn=<EmbeddingBackward>)
torch.Size([3, 4, 5])


In [157]:
n_users = 6
n_items = 9

embedder_users = Embedder(n_users, embedding_dimension)
embedder_items = Embedder(n_items, embedding_dimension)

print("User embeddings")
print(embedder_users.print_weights())

print("Item embeddings")
print(embedder_items.print_weights())

User embeddings
Parameter containing:
tensor([[-0.8669,  0.2444,  0.0309,  0.7737, -1.3752],
        [ 0.5067,  0.2551,  0.7666, -0.6906,  0.1182],
        [-1.4574, -0.7139, -1.2903,  0.1250, -0.1761],
        [-0.7156, -0.6478,  0.4806,  1.3227, -0.5890],
        [-0.0481, -0.7023,  1.6830, -0.0741,  0.9336],
        [ 1.2916, -0.1991,  1.7233, -1.1685,  1.8439],
        [-0.9833,  2.5072,  1.3217,  2.6013,  0.0705]], requires_grad=True)
Item embeddings
Parameter containing:
tensor([[ 0.0167, -0.8347, -0.7065, -0.4941,  0.3440],
        [-0.0895, -1.9731, -0.1074, -0.2042, -0.9967],
        [ 0.0519, -0.4616,  1.5299, -1.5154,  0.2876],
        [-0.2978, -0.1996,  0.5468,  1.8366, -1.0105],
        [ 1.0886,  0.4457, -1.1799, -1.1166,  0.2687],
        [-2.8253,  0.4217,  0.4023, -0.6789, -0.5841],
        [ 0.0880,  0.9356,  1.3416,  0.2956, -0.2526],
        [-0.5941,  0.2293, -0.8367, -0.1605,  0.6026],
        [-1.5620,  0.9095, -0.5343,  0.8031, -1.0756],
        [-0.4596, -0.27

In [158]:
user_item = (1,1)
ratings = [4.5, 2.5, 5.0]
user_1 = torch.tensor(user_item[0], dtype=torch.long)
item_1 = torch.tensor(user_item[1], dtype=torch.long)

In [161]:
user_1_embs = embedder_users(user_1)
print(user_1_embs)
print(user_1_embs.shape)

tensor([ 0.5067,  0.2551,  0.7666, -0.6906,  0.1182],
       grad_fn=<EmbeddingBackward>)
torch.Size([5])


In [160]:
item_1_embs = embedder_items(item_1)
print(item_1_embs)

tensor([-0.0895, -1.9731, -0.1074, -0.2042, -0.9967],
       grad_fn=<EmbeddingBackward>)


## LSTM

Get first sentence

In [196]:
print(input_batch)
sentence1 = input_batch[0]

print(sentence1)

tensor([[ 6, 11,  2,  1],
        [ 9,  4,  7,  1],
        [10,  8,  5,  3]])
tensor([ 6, 11,  2,  1])


In [197]:
print(sentence1.shape)

torch.Size([4])


In [201]:
def prepare_sequence(seq):
    idxs = [word2id[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)

input_ = prepare_sequence(["hola","mundo","cruel", "<PAD>"])
print(input_)
print(input_.shape)

tensor([ 6, 11,  2,  1])
torch.Size([4])


In [211]:
embed = nn.Embedding(vocabulary_size, embedding_dimension)
print(embed(input_))

tensor([[-1.1404e+00, -2.1343e+00,  1.2045e+00, -1.4209e-01,  4.3552e-04],
        [-1.7869e+00, -6.0799e-01, -2.6232e-01,  3.7224e-01,  4.5826e-02],
        [-1.0003e+00, -1.1581e+00,  4.7018e-01, -9.4365e-02, -1.1623e+00],
        [ 2.0146e+00, -4.0179e-01,  5.9712e-01,  4.2814e-01, -5.2815e-01]],
       grad_fn=<EmbeddingBackward>)


In [214]:
word_embeddings = nn.Embedding(vocabulary_size, embedding_dimension)
embeds = embedder(input_)
print(embeds)
print(embeds.shape)

tensor([[ 8.0069e-02, -1.1752e+00, -3.9491e-03,  1.1250e-03, -5.0092e-01],
        [ 5.4267e-01, -1.4515e+00,  5.6311e-01,  3.6198e-01, -8.6320e-01],
        [-9.0353e-01,  6.0791e-02, -7.1391e-01,  1.5493e+00,  2.6345e-03],
        [-7.7714e-01, -2.0969e+00,  1.7835e+00, -2.6246e+00,  4.4385e-02]],
       grad_fn=<EmbeddingBackward>)
torch.Size([4, 5])


In [264]:
hidden_dim = 5
lstm = nn.LSTM(embedding_dimension, hidden_dim)
print("Sentence size:",len(sentence1))
embeds_in = embeds.view(len(sentence1), 1, -1)
print(embeds_in.shape)

Sentence size: 4
torch.Size([4, 1, 5])


In [265]:
lstm_out_, (hn_, cn_) = lstm(embeds_in)
print("lstm_out")
print(lstm_out_)
print(lstm_out_.shape)
print()
print("hn")
print(hn_)
print(hn_.shape)
print("cn")
print(cn_)
print(cn_.shape)

lstm_out
tensor([[[-0.0151,  0.0159,  0.1053, -0.0526,  0.1303]],

        [[-0.0725,  0.0265,  0.1696, -0.1545,  0.1563]],

        [[ 0.0155,  0.0256,  0.0986, -0.1919,  0.2565]],

        [[ 0.0378, -0.0372,  0.2352, -0.1561,  0.0454]]],
       grad_fn=<StackBackward>)
torch.Size([4, 1, 5])

hn
tensor([[[ 0.0378, -0.0372,  0.2352, -0.1561,  0.0454]]],
       grad_fn=<StackBackward>)
torch.Size([1, 1, 5])
cn
tensor([[[ 0.1578, -0.0787,  0.3620, -0.2398,  0.3460]]],
       grad_fn=<StackBackward>)
torch.Size([1, 1, 5])


# Using the whole batch

In [244]:
print(input_batch)
print(input_batch.shape)
embeds_batch_in = embedder(input_batch)
print(embeds_batch_in)
print(embeds_batch_in.shape)

tensor([[ 6, 11,  2,  1],
        [ 9,  4,  7,  1],
        [10,  8,  5,  3]])
torch.Size([3, 4])
tensor([[[ 8.0069e-02, -1.1752e+00, -3.9491e-03,  1.1250e-03, -5.0092e-01],
         [ 5.4267e-01, -1.4515e+00,  5.6311e-01,  3.6198e-01, -8.6320e-01],
         [-9.0353e-01,  6.0791e-02, -7.1391e-01,  1.5493e+00,  2.6345e-03],
         [-7.7714e-01, -2.0969e+00,  1.7835e+00, -2.6246e+00,  4.4385e-02]],

        [[-3.0589e-01, -1.0460e-01,  1.0548e+00,  8.8961e-01,  1.1981e+00],
         [ 5.9098e-01,  3.7616e-01, -3.1290e-01,  5.6046e-01,  1.0438e-01],
         [-5.6224e-02, -5.7728e-01,  1.3235e-01, -1.6854e+00, -5.7394e-01],
         [-7.7714e-01, -2.0969e+00,  1.7835e+00, -2.6246e+00,  4.4385e-02]],

        [[ 1.0106e+00,  1.0115e+00,  3.5873e-01,  9.3127e-01,  1.6046e-01],
         [-3.2366e-01,  7.3624e-01, -5.7591e-01,  4.0074e-01, -1.5145e+00],
         [ 1.2133e+00,  1.3757e+00,  1.1258e+00,  8.7801e-01, -1.1513e+00],
         [-3.7953e-01,  4.6506e-01, -2.4369e+00, -1.8846e-01, 

In [245]:
lstm_out, (hn, cn) = lstm(embeds_in)
print("lstm_out")
print(lstm_out)
print(lstm_out.shape)
print()
print("hn")
print(hn)
print(hn.shape)
print()
print("cn")
print(cn)
print(cn.shape)



lstm_out
tensor([[[ 0.0017,  0.0772, -0.0372,  0.0288,  0.1634]],

        [[ 0.0200,  0.1465, -0.0208, -0.1301,  0.2687]],

        [[-0.0867,  0.0040,  0.0236, -0.1199,  0.2948]],

        [[ 0.0977,  0.1190, -0.0186,  0.1673,  0.0552]]],
       grad_fn=<StackBackward>)
torch.Size([4, 1, 5])

hn
tensor([[[ 0.0977,  0.1190, -0.0186,  0.1673,  0.0552]]],
       grad_fn=<StackBackward>)
torch.Size([1, 1, 5])

cn
tensor([[[ 0.1339,  0.1921, -0.0379,  0.3936,  0.2908]]],
       grad_fn=<StackBackward>)
torch.Size([1, 1, 5])


## Attention

In [246]:
class Attention(nn.Module):

    def __init__(self, dimensions, attention_type='general'):
        super(Attention, self).__init__()

        if attention_type not in ['dot', 'general']:
            raise ValueError("Invalid attention type selected")

        self.attention_type = attention_type
        if self.attention_type == 'general':
            self.linear_in = nn.Linear(dimensions, dimensions, bias=False)

        self.linear_out = nn.Linear(dimensions * 2, dimensions, bias=False)
        self.softmax = nn.Softmax(dim=-1)
        self.tanh = nn.Tanh()

    def forward(self, query, context):
        """

        :param query: [batch_size, output_length, dimensions]
        :param context: [batch_size, output_length, query_length]
        :return:
        output [batch_size, output_legth, dimensions]
        weights [batch_size, output_length, query_length]
        """
        batch_size, output_len, dimensions = query.size()
        query_len = context.size(1)

        if self.attention_type == 'general':
            query = query.reshape(batch_size * output_len, dimensions)
            query = self.linear_in(query)
            query = query.reshape(batch_size, output_len, dimensions)

        # 1. Compute a score for each encoder state
        attention_scores = torch.bmm(query, context.transpose(1,2).contiguous())

        # Compute weights across every context sequence
        attention_scores = attention_scores.view(batch_size * output_len, query_len)
        print("ATT SCORES")
        print(attention_scores.size())

        # 2. Compute the attention weights
        attention_weights = self.softmax(attention_scores)
        attention_weights = attention_weights.view(batch_size, output_len, query_len)

        # 3. Compute the new context vector S
        mix = torch.bmm(attention_weights, context)

        # 4. Concatenate context vector with output of previous time step
        combined = torch.cat((mix, query), dim=2)
        combined = combined.view(batch_size * output_len, 2 * dimensions)

        #
        output = self.linear_out(combined).view(batch_size, output_len, dimensions)
        output = self.tanh(output)

        return mix, output, attention_weights

In [247]:
print("Users embeddings as Query?")
print(user_1_embs)
print(user_1_embs.shape)

Users embeddings as Query?
tensor([ 0.5067,  0.2551,  0.7666, -0.6906,  0.1182],
       grad_fn=<EmbeddingBackward>)
torch.Size([5])


In [266]:
print(lstm_out_.size(0))
aligned_weights = torch.randn(lstm_out_.size(0))
print(aligned_weights)

print("lstm_out_[0]).squeeze()")
x_ = lstm_out_[0].squeeze()
print(x_)
print(x_.shape)

4
tensor([-0.6129, -0.3982, -1.7458,  1.2110])
lstm_out_[0]).squeeze()
tensor([-0.0151,  0.0159,  0.1053, -0.0526,  0.1303],
       grad_fn=<SqueezeBackward0>)
torch.Size([5])


### dot attention

Atencion con el usuario?

In [267]:
for i in range(lstm_out_.size(0)):
    aligned_weights[i] = torch.dot(user_1_embs, lstm_out_[i].squeeze())

In [268]:
print(aligned_weights)
print(aligned_weights.shape)
print(aligned_weights.unsqueeze(0))
print(aligned_weights.unsqueeze(0).shape)

tensor([0.1289, 0.2252, 0.2528, 0.3032], grad_fn=<CopySlices>)
torch.Size([4])
tensor([[0.1289, 0.2252, 0.2528, 0.3032]], grad_fn=<UnsqueezeBackward0>)
torch.Size([1, 4])


In [276]:
aligned_weights_ = F.softmax(aligned_weights.unsqueeze(0))
print(aligned_weights_)
print(aligned_weights_.shape)
print(torch.sum(aligned_weights_.detach(), dim=1))

tensor([[0.2261, 0.2489, 0.2559, 0.2691]], grad_fn=<SoftmaxBackward>)
torch.Size([1, 4])
tensor([1.])


/media/eduardo/SSD_DATA/Appl ML Projects/venv/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  """Entry point for launching an IPython kernel.


Atencion con el item?

In [277]:
for i in range(lstm_out_.size(0)):
    aligned_weights[i] = torch.dot(item_1_embs, lstm_out_[i].squeeze())

In [278]:
print(aligned_weights)
print(aligned_weights.shape)
print(aligned_weights.unsqueeze(0))
print(aligned_weights.unsqueeze(0).shape)

tensor([-0.1604, -0.1883, -0.2789,  0.0313], grad_fn=<CopySlices>)
torch.Size([4])
tensor([[-0.1604, -0.1883, -0.2789,  0.0313]], grad_fn=<UnsqueezeBackward0>)
torch.Size([1, 4])


In [279]:
aligned_weights_ = F.softmax(aligned_weights.unsqueeze(0))
print(aligned_weights_)
print(aligned_weights_.shape)
print(torch.sum(aligned_weights_.detach(), dim=1))



tensor([[0.2456, 0.2388, 0.2181, 0.2975]], grad_fn=<SoftmaxBackward>)
torch.Size([1, 4])
tensor([1.])


/media/eduardo/SSD_DATA/Appl ML Projects/venv/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  """Entry point for launching an IPython kernel.


New Context Vector

In [283]:
print(aligned_weights_.unsqueeze(0).shape)
print(lstm_out_.view(1,-1,hidden_dim).shape)

#context_vector = torch.bmm(aligned_weights.unsqueeze(0), encoder_hidden_state.view(1, -1 ,self.hidden_size))
context_vector = torch.bmm(aligned_weights_.unsqueeze(0), lstm_out_.view(1,-1,hidden_dim))

print(context_vector)
print(context_vector.shape)

torch.Size([1, 1, 4])
torch.Size([1, 4, 5])
tensor([[[-0.0064,  0.0047,  0.1578, -0.1381,  0.1388]]],
       grad_fn=<BmmBackward>)
torch.Size([1, 1, 5])


## Concatenate context_vector, user_emb, item_emb

In [289]:
print(context_vector)
print(context_vector.shape)
print(context_vector[0])
print(context_vector[0].shape)
print()
print(user_1_embs.unsqueeze(0).shape)
print(item_1_embs.unsqueeze(0).shape)

tensor([[[-0.0064,  0.0047,  0.1578, -0.1381,  0.1388]]],
       grad_fn=<BmmBackward>)
torch.Size([1, 1, 5])
tensor([[-0.0064,  0.0047,  0.1578, -0.1381,  0.1388]],
       grad_fn=<SelectBackward>)
torch.Size([1, 5])

torch.Size([1, 5])
torch.Size([1, 5])


In [292]:
x = torch.cat((context_vector[0], user_1_embs.unsqueeze(0), item_1_embs.unsqueeze(0)))
print(x)
print(x.shape)

tensor([[-0.0064,  0.0047,  0.1578, -0.1381,  0.1388],
        [ 0.5067,  0.2551,  0.7666, -0.6906,  0.1182],
        [-0.0895, -1.9731, -0.1074, -0.2042, -0.9967]], grad_fn=<CatBackward>)
torch.Size([3, 5])


In [294]:
print(x.unsqueeze(0))
print(x.unsqueeze(0).shape)


tensor([[[-0.0064,  0.0047,  0.1578, -0.1381,  0.1388],
         [ 0.5067,  0.2551,  0.7666, -0.6906,  0.1182],
         [-0.0895, -1.9731, -0.1074, -0.2042, -0.9967]]],
       grad_fn=<UnsqueezeBackward0>)
torch.Size([1, 3, 5])


In [296]:
x_ = torch.cat((context_vector[0], user_1_embs.unsqueeze(0), item_1_embs.unsqueeze(0)), dim=1)
print(x_)
print(x_.shape)

tensor([[-0.0064,  0.0047,  0.1578, -0.1381,  0.1388,  0.5067,  0.2551,  0.7666,
         -0.6906,  0.1182, -0.0895, -1.9731, -0.1074, -0.2042, -0.9967]],
       grad_fn=<CatBackward>)
torch.Size([1, 15])


In [302]:
x_in = x_.squeeze()
print(x_in)
print(x_in.shape)

tensor([-0.0064,  0.0047,  0.1578, -0.1381,  0.1388,  0.5067,  0.2551,  0.7666,
        -0.6906,  0.1182, -0.0895, -1.9731, -0.1074, -0.2042, -0.9967],
       grad_fn=<SqueezeBackward0>)
torch.Size([15])


In [311]:
n_latent = 32
lin1 = nn.Linear(x_in.shape[0], 32)
print(lin1)

Linear(in_features=15, out_features=32, bias=True)


In [312]:
z1 = lin1(x_)
print(z1)
print(z1.shape)

tensor([[-0.4468,  0.0611,  0.4502,  0.5875, -0.5259, -0.1126,  0.1453,  0.8601,
         -0.1848, -0.5170, -0.9869, -0.3394,  0.5765,  0.0768, -0.3851, -0.2077,
          0.2070, -0.1856,  0.1159,  0.1899,  0.6776, -0.6977, -0.0860, -0.2530,
         -0.8079,  0.1748, -0.2179, -0.2496, -0.4019, -0.4409, -0.3317,  0.5055]],
       grad_fn=<AddmmBackward>)
torch.Size([1, 32])


In [315]:
out= F.relu(z1)
print(out)
print(out.shape)

tensor([[0.0000, 0.0611, 0.4502, 0.5875, 0.0000, 0.0000, 0.1453, 0.8601, 0.0000,
         0.0000, 0.0000, 0.0000, 0.5765, 0.0768, 0.0000, 0.0000, 0.2070, 0.0000,
         0.1159, 0.1899, 0.6776, 0.0000, 0.0000, 0.0000, 0.0000, 0.1748, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.5055]], grad_fn=<ReluBackward0>)
torch.Size([1, 32])


In [316]:
out = out.squeeze()
print(out.shape)
reg = nn.Linear(out.shape[0], 1)
yr1 = reg(out)

torch.Size([32])


In [317]:
print(yr1)
print(yr1.shape)

tensor([0.0258], grad_fn=<AddBackward0>)
torch.Size([1])


Y adjust

In [318]:
y_range = (0, 5.5)
y_adjust= (y_range[1] - y_range[0]) + y_range[0]
print(y_adjust)

5.5


In [322]:
print(yr1 * y_adjust)

tensor([0.1419], grad_fn=<MulBackward0>)


Second head

In [323]:
lin2 = nn.Linear(x_in.shape[0], 32)
print(lin2)

Linear(in_features=15, out_features=32, bias=True)


In [324]:
z2 = lin2(x_)
print(z2)
print(z2.shape)

tensor([[ 0.1459, -0.1948,  0.5146, -0.1132,  0.1092,  0.5377,  0.2426,  0.0366,
          0.0909,  0.5044,  0.2836, -0.4052, -0.0088,  0.0284,  0.1386,  0.4506,
          0.3726,  0.1563,  0.0860,  0.0104,  0.0431,  0.3695, -0.7087, -0.5875,
         -0.0440, -0.2235,  0.2613, -0.8489, -0.0332, -0.3492, -0.2408, -0.0486]],
       grad_fn=<AddmmBackward>)
torch.Size([1, 32])


In [325]:
out= F.relu(z2)
print(out)
print(out.shape)

tensor([[0.1459, 0.0000, 0.5146, 0.0000, 0.1092, 0.5377, 0.2426, 0.0366, 0.0909,
         0.5044, 0.2836, 0.0000, 0.0000, 0.0284, 0.1386, 0.4506, 0.3726, 0.1563,
         0.0860, 0.0104, 0.0431, 0.3695, 0.0000, 0.0000, 0.0000, 0.0000, 0.2613,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000]], grad_fn=<ReluBackward0>)
torch.Size([1, 32])


In [326]:
out = out.squeeze()
print(out.shape)
reg = nn.Linear(out.shape[0], 1)
yr2 = reg(out)

torch.Size([32])


In [327]:
print(yr2)
print(yr2.shape)

tensor([0.0960], grad_fn=<AddBackward0>)
torch.Size([1])


In [328]:
print(yr2 * y_adjust)


tensor([0.5280], grad_fn=<MulBackward0>)
